In [1]:
# Load necessary libraries
library(readr)
library(dplyr)
library(tidytext)
library(tm)
library(topicmodels)
library(ggplot2)
library(parallel) 
library(slam) 
library(Rmpfr)  
library(reshape2) 
library(scales) 
library(grid)
library(ldatuning)
library(textmineR)
library(SnowballC)
library(textstem)
library(textclean)
library(stringr)
library(qdapDictionaries)
library(lexicon)
library(here)
library(lubridate)
library(text2vec)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: NLP


Attaching package: ‘ggplot2’


The following object is masked from ‘package:NLP’:

    annotate


Loading required package: gmp


Attaching package: ‘gmp’


The following objects are masked from ‘package:base’:

    %*%, apply, crossprod, matrix, tcrossprod


C code of R package 'Rmpfr': GMP using 64 bits per limb



Attaching package: ‘Rmpfr’


The following object is masked from ‘package:gmp’:

    outer


The following objects are masked from ‘package:stats’:

    dbinom, dgamma, dnbinom, dnorm, dpois, dt, pnorm


The following objects are masked from ‘package:base’:

    cbind, pmax, pmin, rbind



Attaching package: ‘scales’


The following object is masked from ‘package:readr’:

    col_factor


Loading required package: Matrix


Attaching package: ‘t

In [2]:
i_am("here.txt")
here()

here() starts at /home/ec2-user/SageMaker/esa_analysis



[1] "/home/ec2-user/SageMaker/esa_analysis"

In [3]:
# Read in the original CSV file (replace blank.csv with csv name)
final_corpus <- read.csv(
    here("data", "processed", "clean_original_docs_corpus.csv"))

str(final_corpus)

'data.frame':	5054 obs. of  5 variables:
 $ X    : int  1 2 3 4 5 6 7 8 9 10 ...
 $ GOID : num  4.24e+08 4.24e+08 4.24e+08 4.24e+08 4.24e+08 ...
 $ Title: chr  "BY TOM FERRELL AND MARGOT SLADE; IDEAS AND TRENDS; Pope's Clear Call To Serve Men, Not Their Systems" "In Legal Limbo" "THE CASE FOR THE BITTERING AND BRONX RIVER" "MORE SNAIL DARTERS FOUND IN TENNESSEE" ...
 $ Date : chr  "1980-07-13" "1980-09-07" "1980-10-19" "1980-11-08" ...
 $ Text : chr  "universal bishop roman catholic church pope john paul ii call fulfil preach office bishop end week tour brazil "| __truncated__ "case donald lang murder question legally mr lang unable speak hear read write find guilty chicago kill prostitu"| __truncated__ "bronx river survive ravage time assault civilization year hot dry summer record threaten danger channelization "| __truncated__ "believe population snail darter inch fish demise lead supreme court halt construction tellico dam tennessee yea"| __truncated__ ...


In [4]:
dd <- final_corpus %>% 
    select(c("GOID", "Text"))

colnames(dd) <- c("doc_id", "text")

str(dd)

'data.frame':	5054 obs. of  2 variables:
 $ doc_id: num  4.24e+08 4.24e+08 4.24e+08 4.24e+08 4.24e+08 ...
 $ text  : chr  "universal bishop roman catholic church pope john paul ii call fulfil preach office bishop end week tour brazil "| __truncated__ "case donald lang murder question legally mr lang unable speak hear read write find guilty chicago kill prostitu"| __truncated__ "bronx river survive ravage time assault civilization year hot dry summer record threaten danger channelization "| __truncated__ "believe population snail darter inch fish demise lead supreme court halt construction tellico dam tennessee yea"| __truncated__ ...


In [5]:
s_corpus <- Corpus(DataframeSource(dd))
inspect(s_corpus[1:1])

<<SimpleCorpus>>
Metadata:  corpus specific: 1, document level (indexed): 0
Content:  documents: 1

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [6]:
# Create a Document-Term Matrix using term frequency weighting
dtm_tf <- DocumentTermMatrix(s_corpus, control = list(weighting = weightTf))

# Check the size of the DTM
cat("Number of documents in DTM:", nDocs(dtm_tf), "\n")
cat("Number of terms in DTM:", nTerms(dtm_tf), "\n") 

Warning message in TermDocumentMatrix.SimpleCorpus(x, control):
“custom functions are ignored”


Number of documents in DTM: 5054 
Number of terms in DTM: 44570 


In [7]:
inspect(dtm_tf)

<<DocumentTermMatrix (documents: 5054, terms: 44570)>>
Non-/sparse entries: 1375138/223881642
Sparsity           : 99%
Maximal term length: 42
Weighting          : term frequency (tf)
Sample             :
            Terms
Docs         act endanger environmental federal fish species state water
  2257466370   5        9             0      14    8       7    24     1
  2280955776   1        1             0       3    2       3     4    18
  2831916070   2        2             5       9    1       4     9     3
  2898162467   2        3             0       2    2       3     5    26
  293092531    5       17             2      14   20       9    14     4
  3228154286  19        1             4      17    2       1    37     2
  3228184861  19        1             5      17    2       1    37     2
  422108188    3        1             2       2    1       6    10     4
  427013004    3        2             1       0    0       3     8     2
  520623083    3        6            12      17

In [8]:
# Remove the species from the dtm
dtm_filtered_1 <- dtm_tf[, !colnames(dtm_tf) %in% c("endanger", "species", "act")]

inspect(dtm_filtered_1)

<<DocumentTermMatrix (documents: 5054, terms: 44567)>>
Non-/sparse entries: 1359980/223881638
Sparsity           : 99%
Maximal term length: 42
Weighting          : term frequency (tf)
Sample             :
            Terms
Docs         california environmental federal fish service state time water
  2257466370          1             0      14    8       1    24    8     1
  2280955776          1             0       3    2       3     4   14    18
  2831916070         10             5       9    1       2     9   15     3
  2898162467          1             0       2    2       1     5   18    26
  293092531           9             2      14   20       8    14    8     4
  3228154286          0             4      17    2      11    37    3     2
  3228184861          0             5      17    2      11    37    2     2
  422108188           3             2       2    1       3    10   20     4
  427013004           0             1       0    0      11     8   15     2
  520623083      

In [9]:
# Calculate document frequency (the number of documents each term appears in)
doc_freq <- colSums(as.matrix(dtm_filtered_1) > 0)

# Set the threshold (85% of the documents)
threshold <- 0.85 * nrow(dtm_filtered_1)

# Remove terms that appear in more than 85% of the documents
dtm_filtered_2 <- dtm_filtered_1[, doc_freq <= threshold]

In [10]:
inspect(dtm_filtered_2)

<<DocumentTermMatrix (documents: 5054, terms: 44567)>>
Non-/sparse entries: 1359980/223881638
Sparsity           : 99%
Maximal term length: 42
Weighting          : term frequency (tf)
Sample             :
            Terms
Docs         california environmental federal fish service state time water
  2257466370          1             0      14    8       1    24    8     1
  2280955776          1             0       3    2       3     4   14    18
  2831916070         10             5       9    1       2     9   15     3
  2898162467          1             0       2    2       1     5   18    26
  293092531           9             2      14   20       8    14    8     4
  3228154286          0             4      17    2      11    37    3     2
  3228184861          0             5      17    2      11    37    2     2
  422108188           3             2       2    1       3    10   20     4
  427013004           0             1       0    0      11     8   15     2
  520623083      

In [11]:
empty_docs <- rowSums(as.matrix(dtm_filtered_2)) == 0
print(sum(empty_docs))
empty_doc_ids <- rownames(dtm_filtered_2)[empty_docs]
print(empty_doc_ids)

[1] 0
character(0)


In [12]:
term_frequencies <- slam::col_sums(dtm_filtered_2)
terms_once_across_all <- names(term_frequencies[term_frequencies == 1])
print(terms_once_across_all)

    [1] "albumin"                                   
    [2] "fortaleza"                                 
    [3] "fulfil"                                    
    [4] "hendrie"                                   
    [5] "icejam"                                    
    [6] "metabolize"                                
    [7] "n.r.c."                                    
    [8] "overriding"                                
    [9] "rabbits"                                   
   [10] "reacted"                                   
   [11] "seating"                                   
   [12] "vocation"                                  
   [13] "ballpoint"                                 
   [14] "crestone"                                  
   [15] "germantown"                                
   [16] "griest"                                    
   [17] "gunpoint"                                  
   [18] "haitch"                                    
   [19] "hypluxo"                             

In [13]:
document_counts <- slam::col_sums(dtm_filtered_2 > 0)
terms_in_one_document <- names(document_counts[document_counts == 1])
print(terms_in_one_document)

    [1] "albumin"                                   
    [2] "carnesale"                                 
    [3] "fortaleza"                                 
    [4] "fulfil"                                    
    [5] "hendrie"                                   
    [6] "icejam"                                    
    [7] "insulin"                                   
    [8] "metabolize"                                
    [9] "n.r.c."                                    
   [10] "overriding"                                
   [11] "rabbits"                                   
   [12] "reacted"                                   
   [13] "seating"                                   
   [14] "vocation"                                  
   [15] "ballpoint"                                 
   [16] "crestone"                                  
   [17] "germantown"                                
   [18] "griest"                                    
   [19] "gunpoint"                            

In [14]:
terms_to_remove <- union(terms_once_across_all, terms_in_one_document)
print(terms_to_remove)

    [1] "albumin"                                   
    [2] "fortaleza"                                 
    [3] "fulfil"                                    
    [4] "hendrie"                                   
    [5] "icejam"                                    
    [6] "metabolize"                                
    [7] "n.r.c."                                    
    [8] "overriding"                                
    [9] "rabbits"                                   
   [10] "reacted"                                   
   [11] "seating"                                   
   [12] "vocation"                                  
   [13] "ballpoint"                                 
   [14] "crestone"                                  
   [15] "germantown"                                
   [16] "griest"                                    
   [17] "gunpoint"                                  
   [18] "haitch"                                    
   [19] "hypluxo"                             

In [15]:
dtm_filtered_3 <- dtm_filtered_2[, !(colnames(dtm_filtered_2)%in% terms_to_remove)]
inspect(dtm_filtered_3)

<<DocumentTermMatrix (documents: 5054, terms: 25669)>>
Non-/sparse entries: 1341082/128390044
Sparsity           : 99%
Maximal term length: 22
Weighting          : term frequency (tf)
Sample             :
            Terms
Docs         california environmental federal fish service state time water
  2257466370          1             0      14    8       1    24    8     1
  2280955776          1             0       3    2       3     4   14    18
  2831916070         10             5       9    1       2     9   15     3
  2898162467          1             0       2    2       1     5   18    26
  293092531           9             2      14   20       8    14    8     4
  3228154286          0             4      17    2      11    37    3     2
  3228184861          0             5      17    2      11    37    2     2
  422108188           3             2       2    1       3    10   20     4
  427013004           0             1       0    0      11     8   15     2
  520623083      

In [17]:
terms <- Terms(dtm_filtered_3)
long_terms <- terms[nchar(terms) >= 18]
long_terms

[1] "antidiscrimination"     "disproportionately"     "characteristically"    
 [4] "chlorofluorocarbon"     "unconstitutionally"     "conferencecommittee"   
 [7] "uncharacteristically"   "interconnectedness"     "oversimplification"    
[10] "nonrepresentational"    "nonconfrontational"     "pricewaterhousecoopers"
[13] "hydrofluorocarbons"

In [18]:
empty_docs <- rowSums(as.matrix(dtm_filtered_3)) == 0
print(sum(empty_docs))
empty_doc_ids <- rownames(dtm_filtered_3)[empty_docs]
print(empty_doc_ids)

[1] 0
character(0)


In [19]:
dtm_filtered_4 <- dtm_filtered_3[!empty_docs, ]
inspect(dtm_filtered_4)

<<DocumentTermMatrix (documents: 5054, terms: 25669)>>
Non-/sparse entries: 1341082/128390044
Sparsity           : 99%
Maximal term length: 22
Weighting          : term frequency (tf)
Sample             :
            Terms
Docs         california environmental federal fish service state time water
  2257466370          1             0      14    8       1    24    8     1
  2280955776          1             0       3    2       3     4   14    18
  2831916070         10             5       9    1       2     9   15     3
  2898162467          1             0       2    2       1     5   18    26
  293092531           9             2      14   20       8    14    8     4
  3228154286          0             4      17    2      11    37    3     2
  3228184861          0             5      17    2      11    37    2     2
  422108188           3             2       2    1       3    10   20     4
  427013004           0             1       0    0      11     8   15     2
  520623083      

In [20]:
saveRDS(
    dtm_filtered_4,
    here("data", "final", "clean_dtm.rds")
)